# Training

- Maybe Model defined to only receive one input and not rgb => test out or look into model | but project has 2 color channels
- maybe data generator adjusts data? But does not affect model

In [ ]:
from config import UNetTraining
config = UNetTraining.Configuration()

In [ ]:
# Define callbacks for the early stopping of training, LearningRateScheduler and model checkpointing
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard


checkpoint = ModelCheckpoint(config.model_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = False)

#reduceonplatea; It can be useful when using adam as optimizer
#Reduce learning rate when a metric has stopped improving (after some patience,reduce by a factor of 0.33, new_lr = lr * factor).
#cooldown: number of epochs to wait before resuming normal operation after lr has been reduced.
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.33,
                                   patience=4, verbose=1, mode='min',
                                   min_delta=0.0001, cooldown=4, min_lr=1e-16)

#early = EarlyStopping(monitor="val_loss", mode="min", verbose=2, patience=15)

log_dir = os.path.join('./logs','UNet_{}_{}_{}_{}_{}'.format(timestr,OPTIMIZER_NAME,LOSS_NAME,chs, config.input_shape[0]))
tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=0, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch')

callbacks_list = [checkpoint, tensorboard] #reduceLROnPlat is not required with adaDelta

In [ ]:
# %%time
loss_history = [model.fit(x=X_train_tensor,
                        y=y_train,
                        epochs= config.NB_EPOCHS, 
                        # validation_data=y_train,
                        steps_per_epoch=config.MAX_TRAIN_STEPS,
                        # validation_steps=config.VALID_IMG_COUNT,
                        batch_size=config.BATCH_SIZE,
                        callbacks=callbacks_list
                        # workers=1,
                        # use_multiprocessing=True # the generator is not very thread safe
                )]